# ECMM422  Machine Learning
## Course Assessment 2


This course assessment (CA2) represents 60% of the overall module assessment.

This is an individual exercise and your attention is drawn to the College and University guidelines on collaboration and plagiarism, which are available from the College website.


**Note:**
1. do not change the name of this notebook, i.e. the notebook file has to be named: ca1.ipynb
2. do not remove/delete any cell
3. do not add any cell (you can work on a draft notebook and only copy the function implementations here)
4. do not add you name or student code in the notebook or in the file name

**Evaluation criteria:**

Each question asks for one or more functions to be implemented. 

Each question is awarded a number of marks. 

A (hidden) unit test is going to evaluate if all desired properties of the required function(s) are met. 

If the test passes all the associated marks are awarded, if it fails 0 marks are awarded. The large number of questions and sub-questions allows a fine grading. 


**Efficiency:** There is a cap of a few minutes on the execution of each cell and unit test. Make sure your code is not terribly inefficient (for example having a cell run for hours, e.g. using nested loops rather than NumPy functions that can work directly on  arrays), otherwise the execution of the cell/unit test will be interrupted and considered a failure. 

## Notes:

We call *rank* the number of indices required to get individual elements of an array. A matrix requires two indices (row, column), and has thus rank 2, a vector requires one index and has rank 1, a scalar does not require any index and has rank 0. The components that make up rank are called *axes* (plural of axis). The dimension is how many elements are in a particular axis. A *shape* is a tuple whose length is the rank and elements are the dimension of each axis.

In the rest of the notebook, the term `data matrix` refers to a rank two numpy array where instances are encoded as rows, e.g. a data matrix with 100 rows and 4 columns is to be interpreted as a collection of 100 instances (vectors) each of dimension four.

In the rest of the notebook, the term `vector` refers to a rank one numpy array. When the term `distance` is used we mean the Euclidean distance. 

The functions you are required to write need to take in input and return as output such objects (i.e. not python lists).

---

When a required function can be implemented directly by a library function it is intended that the candidate should write her own implementation of the function.

---

Do not assume that the implementations provided in the Workshops exercises contain no mistakes. You should write and are ultimately responsible for the code that you submit in this Assessment.

---

You are asked to solve the quadratic optimization problem using the library [cvxopt](http://cvxopt.org/).  

You can install the library via:

`pip install cvxopt`

or

`conda install -c conda-forge cvxopt`

Then you should be able to import the module with:

    `import cvxopt`
  

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import cvxopt

## Question 1  [marks 3]


a) Make the following functions: 

- `TPR = true_positive_rate(preds, targets)` 
- `FPR = false_positive_rate(preds, targets)` 

to compute the sequence of true (resp. false) positive rates for all possible threshold values.
The functions should work equally well when the input is of the type `list` or `numpy array`. 
The input `preds` contains the predictions as float numbers and the input `targets` contains the true class labels encoded as the integers `1` or `-1`. 

b) Make the function `compute_auc(TPR, FPR)` to compute the area under the curve of the ROC given in input the list of true poritive rates and false positive rates for all possible threshold values.

In [ ]:
def true_positive_rate(preds, targets):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
def false_positive_rate(preds, targets):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
def compute_auc(TPR, FPR):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2  [marks 3]


a) Make a function `classifier = train_rf(X_train, y_train, param)` that takes in input a data matrix `X_train`, an associated target array `y_train`, and a single scalar parameter `param` which corresponds to the maximum depth of the decision trees used in the forest, and returns a fit classifier object of type `sklearn.ensemble.RandomForestClassifier`. In this implementation you must fix the number of decision trees used in the forest to the value 10.

b) Make a function `preds = test_rf(X_test, classifier)` that takes in input a data matrix `X_test`, a classifier object `classifier` and returns the prediction array `preds`.

c) Make a function `scores = score_rf(X_test, classifier)` that takes in input a data matrix `X_test`, a classifier object `classifier` and returns the array `scores` containing the probability for each instance to belong to the positive class (which is assumed to be encoded with the integer `1`).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def train_rf(X_train, y_train, param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
def test_rf(X_test, estimator):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
def score_rf(X_test, estimator):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3  [marks 2]

Make a function `scores_list = compute_scores(train_func, score_func, param, X_train, y_train, X_test, num)` that trains a classifier on `X_train, y_train` and tests it on `X_test` a number of times equal to `num`. The function outputs a list of score arrays of size `num`, and each score array contains the predicted probability for each instance in `X_test` to belong to the positive class (which is assumed to be encoded with the integer `1`). Note that by using a randomized classifier (like the random forest) you will get different output score arrays each time. 

In [ ]:
def compute_scores(train_func, score_func, param, X_train, y_train, X_test, num):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 4  [marks 7]

Make a function `tpr_low, tpr_high, tpr_mid, fpr = compute_tpr_fpr_range(scores_list, y_test, false_positive_rate_func, true_positive_rate_func, low_quantile, high_quantile)`. The function takes in input a list of predicted scores, the true classes, a function to compute the false positive rate and one to compute the true positive rate and two threshold values: low_quantile and high_quantile. The function will output 4 lists (or arrays): 

- tpr_low: the true positive rate corresponding to the `low_quantile` true positive rate 
- tpr_high: the true positive rate corresponding to the `high_quantile` true positive rate
- tpr_mid: the true positive rate corresponding to the median true positive rate
- fpr: the false positive rate

Note: instead of a single pair `(true positive rate,false positive rate)` the `scores_list` allows you to have mutiple true positive rate values for the same false positive rate value, i.e. a distribution of true positive rate values; the function `compute_tpr_fpr_range` must output the values corresponding to the quantiles of such distributions (e.g. the true positive rate corresponding to the 95 percentile of the true positive rate distribution) for each corresponding false positive rate value (see the figure the Checkpoint cell below).


In [ ]:
def compute_tpr_fpr_range(scores_list, y_test, false_positive_rate_func, true_positive_rate_func, low_quantile, high_quantile):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 5  [marks 3]

Make a function `plot_roc(tpr_low, tpr_high, tpr_mid, fpr, compute_auc_func)` that takes in input the true positive rate corresponding to the `low_quantile` true positive rate, the true positive rate corresponding to the `high_quantile` true positive rate, the true positive rate corresponding to the median true positive rate, the false positive rate and a function to compute the ROC AUC and produces a plot like:

<img src='plot1.png' width=300>

In [ ]:
def plot_roc(tpr_low, tpr_high, tpr_mid, fpr, compute_auc_func):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Checkpoint

This is just a check-point, i.e. it is for you to see that you are correctly implementing all functions. 

Execute the following code (just execute the next cell):
```python
from sklearn.datasets import make_classification
X,y = make_classification(n_samples=500, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.1, class_sep=1.0, random_state=2)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=50, random_state=2)

scores_list = compute_scores(train_rf, score_rf, 5, X_train, y_train, X_test, num=100)
tpr_low, tpr_high, tpr_mid, fpr = compute_tpr_fpr_range(scores_list, y_test, false_positive_rate, true_positive_rate, low_quantile=0.05, high_quantile=0.95)
plot_roc(tpr_low, tpr_high, tpr_mid, fpr, compute_auc)
```

You should obtain something like:

<img src='plot1.png' width=300>

In [ ]:
# Just run the following code, do not modify it

from sklearn.datasets import make_classification
X,y = make_classification(n_samples=500, n_features=20, n_informative=2, n_redundant=2, n_repeated=0, n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.1, class_sep=1.0, random_state=2)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=50, random_state=2)

scores_list = compute_scores(train_rf, score_rf, 5, X_train, y_train, X_test, num=100)
tpr_low, tpr_high, tpr_mid, fpr = compute_tpr_fpr_range(scores_list, y_test, false_positive_rate, true_positive_rate, low_quantile=0.05, high_quantile=0.95)
plot_roc(tpr_low, tpr_high, tpr_mid, fpr, compute_auc)

## Question 6  [marks 6]

Solve a constrained quadratic program (QP) with `cvxopt`. 

The standard form for a QP in `cvxopt` notation is:

\begin{align*}
        \min_{x}  \tfrac{1}{2} x^T P x &+ q^T x \\
        \text{such that   }\quad  Gx &\preceq h\\
        Ax & = b
\end{align*}
where $Gx \preceq h$ means that the inequality is taken element-wise for each row of $Gx$ and $h$.

In `cvxopt` the QP is defined by the parameters $P, q, G, h, A$ and $b$.  The matrix $P$ and the vector $q$ must be specified, but the others are optional.   Note that all optimisations must be manipulated to be of this form.

Consider this problem:

\begin{align*}
        \min_{x}  \tfrac{1}{8} x^2 - \frac{1}{10} y ^ 2+ & \frac{1}{4} x + y\\
        \text{such that   }\quad y - x & \le 0\\
        x + 3y & \ge 15\\
        3x + 5y & \ge 40\\
        \frac{1}{2} x + 4y & \le 50
\end{align*}

Consider a grid covering the range $[0, 15] \times [-2, 15]$ (i.e. $x \in [0, 15]$ and $y \in [-2, 15]$).

Make a function `P,q,G,h = make_matrices()` that returns the corresponding matrices of type `cvxopt.matrix` for this specific problem.

In [ ]:
from cvxopt import matrix

def make_matrices():
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 7  [marks 4]

Make a function `sol, obj = compute_solution_and_objective(P,q,G,h)` that takes in input the corresponding matrices of type `cvxopt.matrix` and returns `sol` containing the vector (as a numpy array) that minimizes the constrained QP problem and a scalar `obj` that contains the value of the objective function evaluated at the solution (i.e. the minimum of the objective function).

In [ ]:
from cvxopt import solvers

def compute_solution_and_objective(P,q,G,h):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 8  [marks 6]

Make a function `plot_solution(sol)` that takes in input the solution array `sol` to the previous QP problem and displays:

- the feasibility region by representing the constraints 
- the objective function via colored contour lines and an associated color scale 
- the solution as a red dot

The result (applied to a different QP problem) should resemble in style the following plot:

<img src='plot2b.png' width=300>

In [ ]:
def plot_solution(sol):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 


## Checkpoint

This is just a check-point, i.e. it is for you to see that you are correctly implementing all functions.

Execute the following code (just execute the next cell):
```python
P,q,G,h = make_matrices()
sol, obj = compute_solution_and_objective(P,q,G,h)
plot_solution(sol)
print(sol, obj)
```

If you had to solve the following **different** problem:
\begin{align*}
        \min_{x}  \tfrac{1}{2} x^2 + & 3x + 4y\\
        \text{such that   }\quad x, y & \ge 0\\
        x + 3y & \ge 15\\
        2x + 5y & \le 100\\
        3x + 4y & \le 80
\end{align*}
in the domain $[-5, 20] \times [-5, 20]$, then you would obtain a picture like this:

<img src='plot2.png' width=300>

Note however that **your problem is different** and you will obtain a different figure and a different solution.

In [ ]:
# Just run the following code, do not modify it

P,q,G,h = make_matrices()
sol, obj = compute_solution_and_objective(P,q,G,h)
plot_solution(sol)
print(sol, obj)

## Kernelized Large Margin Classifiers

In this exercise you are asked to develop the code to solve the optimization problem that defines a kernelized Large Margin Classifier with soft margins. 

### Linear case

We need to match the solver's API which, according to the documentation is of the form:

\begin{aligned}
    & \min \frac{1}{2} x^TPx + q^Tx
    \\
     s.t. \ & \ Gx \leq h 
    \\
    & \ Ax = b
\end{aligned}

Recall that the dual problem is expressed as:

\begin{aligned}
\max_{\alpha} \sum_i^m \alpha_i - \frac{1}{2} \sum_{i,j}^m y^{(i)}y^{(j)} \alpha_i \alpha_j <x^{(i)}, x^{(j)}>
\end{aligned}

Let $\mathbf{H}$ be a matrix such that $H_{i,j} = y^{(i)}y^{(j)} <x^{(i)} x^{(j)}>$, then the optimization becomes:

\begin{aligned}
    & \max_{\alpha} \sum_i^m \alpha_i  - \frac{1}{2}  \alpha^T \mathbf{H}  \alpha
    \\
     s.t. & \ \alpha_i \geq 0 
    \\
    &  \ \sum_i^m \alpha_i y^{(i)} = 0  
\end{aligned}

We convert the sums into vector form and multiply both the objective and the constraint by −1 which turns this into a minimization problem and reverses the inequality

\begin{aligned}
    & \min_{\alpha}  \frac{1}{2}  \alpha^T \mathbf{H}  \alpha - 1^T \alpha
    \\
    & s.t. \ - \alpha_i \leq 0 
    \\
    & s.t. \ y^T \alpha = 0 
\end{aligned}

We are now ready to convert our numpy arrays into the cvxopt format, using the same notation as in the documentation this gives

* $P:=H$ a matrix of size $m×m$
* $q:=−\vec{1}$  a vector of size $m×1$
* $G:=−diag[1]$ a diagonal matrix of -1s of size $m×m$
* $h:=\vec{0}$ a vector of zeros of size $m×1$
* $A:=y^T$ the label vector of size $1×m$
* $b:=0$ a scalar

Note that in the simple example of $m=2$ the matrix $G$ and vector $h$ which define the constraint are 

$$G = \begin{bmatrix} -1 & 0 \\ 0 & -1 \end{bmatrix} \ \ \text{and} \ \ h = \begin{bmatrix} 0 \\ 0  \end{bmatrix}$$

### Computing the matrix $\mathbf{H}$ in vectorized form

Consider the simple example with 2 input samples $\{x^{(1)}, x^{(2)}\} \in \mathbb{R}^2$ which are two dimensional vectors. i.e. $x^{(1)} = (x_1^{(1)} , x_2^{(1)})^T$

$$X = \begin{bmatrix} x_1^{(1)} & x_2^{(1)} \\ x_1^{(2)} & x_2^{(2)} \end{bmatrix} \ \ \text{and} \ \ y = \begin{bmatrix} y^{(1)}  \\ y^{(2)} \end{bmatrix}$$

We now proceed to creating a new matrix $X′$ where each input sample $x$ is multiplied by the corresponding output label y. This can be done easily in Numpy using vectorization and padding.

$$X' = \begin{bmatrix} x^{(1)}_1 y^{(1)} & x^{(1)}_2y^{(1)} \\
x^{(2)}_1y^{(2)} & x^{(2)}_2y^{(2)} \end{bmatrix}$$

Finally we take the **matrix multiplication** of $X′$ and its transpose giving $H=X′X′^T$

$$H = X' @ X'^T = \begin{bmatrix} x^{(1)}_1 y^{(1)} & x^{(1)}_2y^{(1)} \\
x^{(2)}_1y^{(2)} & x^{(2)}_2y^{(2)} \end{bmatrix} \begin{bmatrix} x^{(1)}_1 y^{(1)} & x^{(2)}_1 y^{(2)}  \\
x^{(1)}_2y^{(1)} & x^{(2)}_2y^{(2)} \end{bmatrix}$$

$$H = \begin{bmatrix}  x^{(1)}_1 x^{(1)}_1y^{(1)}y^{(1)} + x^{(1)}_2x^{(1)}_2y^{(1)}y^{(1)} & x^{(1)}_1 x^{(2)}_1y^{(1)}y^{(2)} + x^{(1)}_2x^{(2)}_2y^{(1)}y^{(2)} \\ x^{(2)}_1 x^{(1)}_1y^{(2)}y^{(1)} + x^{(2)}_2x^{(1)}_2y^{(2)}y^{(1)} & x^{(2)}_1 x^{(2)}_1y^{(2)}y^{(2)} + x^{(2)}_2x^{(2)}_2y^{(2)}y^{(2)} \end{bmatrix}$$


### Soft margin
For the softmax margin SVM, recall that the optimization problem can be expressed as

\begin{aligned}
    & \max_{\alpha} \sum_i^m \alpha_i  - \frac{1}{2}  \alpha^T \mathbf{H}  \alpha
    \\
     s.t. \ \ & 0 \leq \alpha_i \leq C 
    \\
    &  \ \sum_i^m \alpha_i y^{(i)} = 0  
\end{aligned}

which can be written in standard form as

\begin{aligned}
    & \min_{\alpha}  \frac{1}{2}  \alpha^T \mathbf{H}  \alpha - 1^T \alpha
    \\
     s.t. & \ - \alpha_i \leq 0 
    \\
     & \alpha_i \leq C
     \\
     &\ y^T \alpha = 0  
\end{aligned}

This is almost the same problem as previously, except for the additional inequality constraint on $\alpha$. We translate this new constraint into standard form by concatenating below matrix $G$ a diagonal matrix of 1s of size $m \times m$. Similarly for the vector $h$ to which the value of $C$ is added $m$ times.

Note that in the simple example of $m=2$ the matrix $G$ and vector $h$ which define the constraint are

$$G = \begin{bmatrix} -1 & 0 \\ 0 & -1 \\ 1 & 0 \\ 0 & 1\end{bmatrix} \ \ \text{and} \ \ h = \begin{bmatrix} 0 \\ 0 \\ C \\ C \end{bmatrix}$$

### Non linear case

What has been written above corresponds to the hard and the soft margin case for a linear large margin classification problem.  

You now need to appy the *kernel trick*, i.e. you need to replace the dot product operations in the input space $<x^{(i)} x^{(j)}>$ with the dot product in the feature space via a kernel function $K(x^{(i)},x^{(j)}) = <\phi(x^{(i)}) \phi(x^{(j)})>$. 

## Question 9  [marks 5]

Make the function `H = make_H_k(X,t, kernel_function, param)` to compute the matrix $H$ for a data matrix `X` and an associated target vector `t` given a kernel function `kernel_function` with parameter `param`. 

In [ ]:
def make_H_k(X,t, kernel_function, param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 10  [marks 3]

Make the function `P, q, G, h, A, b = setup_optimization_soft_k(X, t, C, kernel_function, param)` to compute the necessary matrices to solve the QP problem associated to the soft margin kernelized classification task using the library [cvxopt](http://cvxopt.org/). The function receives in input a data matrix `X`, an associated target vector `t`, a regularization parameter `C` and a kernel function `kernel_function` with parameter `param`. 

In [ ]:
def setup_optimization_soft_k(X,t,C, kernel_function, param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 11  [marks 1]

Make the function `support_vectors, support_targets, support_alphas, support_ids = compute_support_vectors(X,t,alphas)`  to compute:
- `support_vectors` the support vectors
- `support_targets` the target vector associated to the support vectors
- `support_alphas` the vector of the dual variables $\alpha_i$ associated to the support vectors
- `support_ids` a boolean vector that indicates if a position is associated to a support vector (w.r.t. the original data matrix) (i.e. if the first instance is a support vector and the second one it is not, then `support_ids` = [True, False])

In [ ]:
def compute_support_vectors(X,t,alphas):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 12  [marks 2]

Make the function `model = train_slmkc(X, t, C, kernel_function, param)` to compute the necessary elements to represent a kernelized soft large margin classifier trained over an input data matrix `X` with an associated target vector `t`, for a regularization parameter `C` and a kernel function `kernel_function` with parameter `param`. 

Note: you may implement `model` as a tuple containing information on the support vectors and the dual variables, i.e. `model = (support_vectors,support_targets,support_alphas, support_ids)`.

In [ ]:
def train_slmkc(X, t, C, kernel_function, param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 13  [marks 7]

Given the kernelized soft large margin discriminant function of the form:
$$ y(x) = \sum_{n \in SV} \alpha_n t_n K(x_n,x) + b $$

and given the encoded `model` to represent the necessary elements for a trained kernelized soft large margin classifier

make the function `b = compute_bias(kernel_function, param, C, model)` to compute the offset/bias term `b`,  for a regularization parameter `C` and a kernel function `kernel_function` with parameter `param`. 

In [ ]:
def compute_bias(kernel_function, param, C, model):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 14  [marks 7]

Make the function `scores = score_slmkc(X_test, kernel_function, param, C, model)` to compute the scores for the kernelized soft large margin discriminant function over the data matrix `X_test`, for a regularization parameter `C`, a kernel function `kernel_function` with parameter `param` and an encoded model `model`.

In [ ]:
def score_slmkc(X_test, kernel_function, param, C, model):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 15  [marks 2]

Make the function `preds = test_slmkc(X_test, kernel_function, param, C, model)` to predict the class label for the kernelized soft large margin discriminant function over the data matrix `X_test`, for a regularization parameter `C`, a kernel function `kernel_function` with parameter `param` and an encoded model `model`.

In [ ]:
def test_slmkc(X_test, kernel_function, param, C, model):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
# this function is provided to you, you do not need to alter it in any way.

class MySoftLargeMarginKernelClassfier(object):
    def __init__(self, C, kernel_function, param):
        self.kernel_function = kernel_function
        self.param = param
        self.C = C
        self.support_vectors = None
        self.support_targets = None
        self.support_alphas = None
        self.support_ids = None
    
    def fit(self, X_train, y_train):
        self.model = train_slmkc(X_train, y_train, self.C, self.kernel_function, self.param)
        self.support_vectors, self.support_targets,self.support_alphas, self.support_ids = self.model
        return self
    
    def predict(self, X_test):
        return test_slmkc(X_test, self.kernel_function, self.param, self.C, self.model)
        
    def decision_function(self, X_test):
        return score_slmkc(X_test, self.kernel_function, self.param, self.C, self.model)
    
    def predict_proba(self, X_test):
        scores = self.decision_function(X_test)
        probs = 1 / (1 + np.exp(-scores))
        return probs

## Question 16  [marks 2]

Make the function `linear_kernel_function(x1,x2,param=None)` corresponding to the linear kernel defined as:
$$ K(x_1,x_2) =  x_1^T x_2 $$
where $x_1$ and $x_2$ are vectors. 

In [ ]:
def linear_kernel_function(x1,x2,param=None):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 17  [marks 3]

Make the function `polynomial_kernel_function(x1,x2,param)` corresponding to the polynomial kernel defined by the parameter $r=$`param`:
$$ K(x_1,x_2) =  (1 + x_1^T x_2)^r$$
where $x_1$ and $x_2$ are vectors. 

In [ ]:
def polynomial_kernel_function(x1,x2,param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
# this function is provided to you, you do not need to alter it in any way.

def plot(X, y, support_vectors=None, predict_func=None):
    cmap='bwr'
    res = 100
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    delta = max((x_max-x_min)/5, (y_max-y_min)/5)
    x_min -= delta
    y_min -= delta
    x_max += delta
    y_max += delta
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, res),np.linspace(y_min, y_max, res))
    if predict_func is None:
        Z = np.ones(xx.shape)*.5
    else:
        Z = predict_func(np.hstack([xx.reshape(-1,1), yy.reshape(-1,1)]))
        if Z.ndim == 1:
            Z = Z.reshape(xx.shape)
        else:
            Z = Z[:,-1].reshape(xx.shape)
    if support_vectors is None:
        edge_colors = 'w'
        sizes = 50
    else:
        edge_colors = ['k' if s else 'w' for s in support_vectors]
        sizes = [100 if s else 50 for s in support_vectors]
    plt.figure(figsize=(5,5))
    plt.contourf(xx, yy, Z, levels=20, cmap=cmap, alpha=.5)
    plt.scatter(X[:,0],X[:,1], c=y, s=sizes, edgecolors=edge_colors, cmap=cmap)
    plt.grid()
    plt.axis('equal')
    plt.show()

## Checkpoint

This is just a check-point, i.e. it is for you to see that you are correctly implementing all functions. 

Execute the following code (just execute the next cell):

```python
from sklearn.datasets import make_classification
X,y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.0, class_sep=1.0, random_state=5)
y[y==0]=-1

est = MySoftLargeMarginKernelClassfier(C=1e2, kernel_function=linear_kernel_function, param=None).fit(X, y)
plot(X,y, support_vectors=est.support_ids, predict_func=est.predict_proba)

est = MySoftLargeMarginKernelClassfier(C=1e-2, kernel_function=polynomial_kernel_function, param=5).fit(X, y)
plot(X,y, support_vectors=est.support_ids, predict_func=est.predict_proba)
```

You should obtain something like:

<img src='plot3.png' width=300>

In [ ]:
# Just run the following code, do not modify it

from sklearn.datasets import make_classification
X,y = make_classification(n_samples=100, n_features=2, n_informative=2, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=2, weights=None, flip_y=0.0, class_sep=1.0, random_state=5)
y[y==0]=-1

est = MySoftLargeMarginKernelClassfier(C=1e2, kernel_function=linear_kernel_function, param=None).fit(X, y)
plot(X,y, support_vectors=est.support_ids, predict_func=est.predict_proba)

est = MySoftLargeMarginKernelClassfier(C=1e-2, kernel_function=polynomial_kernel_function, param=5).fit(X, y)
plot(X,y, support_vectors=est.support_ids, predict_func=est.predict_proba)

## Question 18  [marks 1]


a) Make a function `estimator = train_svm(X_train, y_train, param)` that takes in input a data matrix `X_train`, an associated target array `y_train`, and a single scalar parameter `param` which corresponds to the regularization parameter of the linear Support Vector machine classifier, and returns a fit classifier object of type `sklearn.svm.LinearSVC`.

b) Make a function `preds = test_svm(X_test, classifier)` that takes in input a data matrix `X_test`, a classifier object `classifier` and returns the prediction array `preds`.

c) Make a function `scores = score_svm(X_test, classifier)` that takes in input a data matrix `X_test`, a classifier object `classifier` and returns the array `scores` containing the distance from the decision surface  for each instance.

In [ ]:
from sklearn.svm import LinearSVC

def train_svm(X_train, y_train, param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
def test_svm(X_test, estimator):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [ ]:
def score_svm(X_test, estimator):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 19  [marks 2]


The one-vs-one strategy constructs one classifier per pair of classes. At prediction time, the class which received the most votes is selected. 

In the event of a tie (among all those cases that achieve an equal number of votes), it selects the class with the highest aggregate classification confidence by summing over the pair-wise classification confidence levels computed by the underlying binary classifiers.

Make the function `estimators = train_OvO(X_train, y_train, train_func, param)` that takes in input the data matrix `X_train`, the target vector `y_train`, the training procedure `train_func` with an associated parameter `param` and it outputs an object that represent the estimators fit using the OvO strategy.

In [ ]:
def train_OvO(X_train, y_train, train_func, param):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 20  [marks 7]

Make the function `preds = test_OvO(X_test, test_func, score_func, estimators)` that takes in input the data matrix `X_test`, the classification procedure `test_func`, the score function `score_func`, the estimators object and it returns the predicted class for each instance in the data matrix using the OvO strategy.

Note: only when two or more classifiers end up in a tie the algorithm should resort to the score to break the tie. 

In [ ]:
def test_OvO(X_test, test_func, score_func, estimators):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 21  [marks 4]

Make the function `confusion_table(targets, preds)` to output the confusion matrix as a `n x n` Numpy array where `n` is the number of classes. The confusion table is a matrix whose i-th row and j-th column entry indicates the number of samples with true label being the i-th class and predicted label being the j-th class.

As an example, when executing the following code:

```python
test_targets = np.array([0,1,2,0,1,2])
test_preds  =  np.array([0,1,2,0,0,0])
print(confusion_table(test_targets, test_preds))
```
you should obtain:
```
array([[2, 0, 0],
       [1, 1, 0],
       [1, 0, 1]])
```

In [ ]:
def confusion_table(targets, preds):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Cross correlations

In the following section you are asked to work on the notion of cross correlations in 2 and 3 dimensions.

Since the computation of a cross correlation becomes undefined at those positions that are very close to the image boundaries, you are asked to **pad** the original signal, i.e. to extend the signal using a specific strategy.

Note that the resulting cross correlation has the same size as the original signal. 

## Question 22  [marks 6]

Make the function `Mp = padding_repeating(M, length)` that takes in input a 2D numpy array `M` and a scalar `length` and pads `M` with `length` replicas of the boundary cells. 

For example running the following code 
```python
n=5
M=np.arange(n*n).reshape(n,n)
print(M)
length=1
print(length)
print(padding_repeating(M, length))
length=2
print(length)
print(padding_repeating(M, length))
```

yields

```
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
1
[[ 0  0  1  2  3  4  4]
 [ 0  0  1  2  3  4  4]
 [ 5  5  6  7  8  9  9]
 [10 10 11 12 13 14 14]
 [15 15 16 17 18 19 19]
 [20 20 21 22 23 24 24]
 [20 20 21 22 23 24 24]]
2
[[ 0  0  0  1  2  3  4  4  4]
 [ 0  0  0  1  2  3  4  4  4]
 [ 0  0  0  1  2  3  4  4  4]
 [ 5  5  5  6  7  8  9  9  9]
 [10 10 10 11 12 13 14 14 14]
 [15 15 15 16 17 18 19 19 19]
 [20 20 20 21 22 23 24 24 24]
 [20 20 20 21 22 23 24 24 24]
 [20 20 20 21 22 23 24 24 24]]
```

In [ ]:
def padding_repeating(M, length):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 23  [marks 4]

Make the function `cross_correlation2D(image, kernel, padding_func)` that takes in input a $m \times n$ 2D array `image` and a square $k \times k$ 2D array `kernel` with $k$ an odd number (e.g. kernel is a 3x3 or 5x5, etc, matrix), and a padding function `padding_func`. The function outputs an array of the same size $m \times n$ as `image` which corresponds to the 2D cross correlation between the padded `image` and `kernel`.

In [ ]:
def cross_correlation2D(image, kernel, padding_func):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 24  [marks 3]

Make the function `padding_zeros_3D(M, length)` that takes in input a 3D numpy array `M` and a scalar `length` and pads `M` with with replicas of 0 entries along all sides.

For example:
```python
padding_zeros_3D(np.ones((3,3,3)), length=1)
```
yields
```
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])
```

In [ ]:
def padding_zeros_3D(M, length):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 25  [marks 3]

Make the function `cross_correlation3D(image, kernel, padding_func)` that takes in input a $m \times n \times p$ 3D array `image` and a square $k \times k \times k$ 3D array `kernel` with $k$ an odd number (e.g. kernel is a 3x3x3 or 5x5x5, etc, matrix), and a padding function `padding_func`. The function outputs an array of the same size $m \times n \times p$ as `image` which corresponds to the 3D cross correlation between the padded `image` and `kernel`.

In [ ]:
def cross_correlation3D(image, kernel, padding_func):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 26  [marks 2]

Make the function `make_averaging_kernel(n_dim, length)` that generates a kernel with `n_dim` dimensions (e.g. n_dim = 3) of shape $(l \times l \times \ldots l)$ with $l=$`lenght`. The function should build a specific kernel that, when used in a cross correlation operation, computes the average of all the values in its field of action (e.g. a 3x3x3 kernel makes the average of the 27 neighboring cells).

In [ ]:
def make_averaging_kernel(n_dim, length):
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This cell is reserved for the unit tests. Do not consider this cell. 


## Checkpoint

This is just a check-point, i.e. it is for you to see that you are correctly implementing all functions. 

Note that a color image is encoded as a 3D array, where the 3rd dimension is used to encode the color information.

Execute the following code (just execute the next cell):

```python
from PIL import Image
image = np.array(Image.open('fish.jpeg')).astype(int)

# original image
print(image.shape)
plt.imshow(image)
plt.show()

# correlation
kernel = make_averaging_kernel(n_dim=3, length=7)
out = cross_correlation3D(image, kernel, padding_zeros_3D)
out = out.astype(int)
plt.imshow(out)
plt.show()
```

You should obtain something like:
```
(510, 510, 3)
```

<img src='plot4.png' width=300>

In [ ]:
# Just run the following code, do not modify it

from PIL import Image
image = np.array(Image.open('fish.jpeg')).astype(int)

# original image
print(image.shape)
plt.imshow(image)
plt.show()

# correlation
kernel = make_averaging_kernel(n_dim=3, length=5)
out = cross_correlation3D(image, kernel, padding_zeros_3D)
out = out.astype(int)
plt.imshow(out)
plt.show()